In [20]:
import load_data as ld
import numpy as np
import pandas as pd
from eunjeon import Mecab
from soynlp.normalizer import *
from sklearn.feature_extraction.text import CountVectorizer
import re   

def clean_text(str):
    txt = re.sub('[-=+,#/\?:^@*\"※~ㆍ!』‘|\(\)\[\]`\'…》\”\“\’·]', ' ', str)
    txt = only_text(txt)
    return txt

def get_nouns(str):   # 형용사 등도 이용할 수 있도록
    tagger = Mecab()
    nouns_list = tagger.nouns(str)
    nouns_str = ' '.join(nouns_list)
    return nouns_str

def get_stcs_nouns(ListOfSentence):
    return list(map(get_nouns, ListOfSentence))
    

def para2stcs(paragraph):
    return paragraph.split('. ')

def cal_tfidf(paragraphs):  # TF-IDF뿐만 아니라 textrank 등도 이용할 수 있도록
    vectorizer = CountVectorizer()
    dtm = vectorizer.fit_transform(paragraphs)  # Document Term Matrix
    
    tf = pd.DataFrame(dtm.toarray(), columns = vectorizer.get_feature_names())  # Term Freqeuncy
    df = tf.astype(bool).sum(axis = 0)  # Document Frequency 

    D = len(tf)         # 문서 개수
    idf = np.log((D+1) / (df+1)) + 1    # Inverse Document Frequency
    
    # TF-IDF
    tfidf = tf * idf                      
    tfidf = tfidf / np.linalg.norm(tfidf, axis = 1, keepdims = True)
    
    # change the form of dataframe
    tfidf = tfidf.T
    tfidf.columns = ['tfidf']
    return tfidf.sort_values(by='tfidf')

def preprocessing(paragraphs):  # paragraphs : ["문단1", "문단2", ...]
    
    clean_paras = list(map(clean_text, paragraphs))
    sentences = list(map(para2stc, clean_paras)) # sentences : [["문단1의 문장1", "문단1의 문장2"], ["문단2의 문장1", ...], ...]
    nouns_paras = list(map(get_nouns, clean_paras))
    tfidf = cal_tfidf(nouns_paras)
    return nouns_paras, tfidf, sentences
    

In [10]:
paragraphs = ld.read_txt("감초.txt")     # [ 문단1, 문단2, ...]

In [21]:

clean_paras = list(map(clean_text, paragraphs))
clean_stcs = list(map(para2stcs, clean_paras))
nouns_paras = list(map(get_nouns, clean_paras))
nouns_stcs = list(map(get_stcs_nouns, clean_stcs))

In [22]:
nouns_stcs

[['쌍떡잎식물 장 미목 콩과 여러해살이풀',
  '말 풀',
  '우리 나라 아시아 감초 종 유럽 미국 우랄 감초 중국 감초 약재 유럽 미국 리 코리스 종',
  '한국 민감 초 유럽 감초 약재 사탕 과자 생산',
  '가지 종 감초 종 주의 필요',
  '서양 리 코리스 이름 리큐르 어원',
  '애초 리큐르 약용 술 의미 거 감초',
  '뿌리 채취 한약재 감미료 등 사용',
  '감초 안 약용 성분 다수 질환 효과',
  '약방 감초 속담 정도',
  '옛날 이야기 약방 아내 병자 감초 병',
  '플라시보 효과 수 병 효과 뜻',
  '감초 뿌리 옆 것 밑 것 이중 밑 것 약재 사용 옆 것 가을 채취 봄 등',
  '약재 직접 효능 위장 보호 독성 중화',
  '약재 독 중화 효능 완화 효능 배합 역할',
  '맛 탕약 맛 역할',
  '한방 약재 처방 때 비교',
  '약방 감초 속담 이것 비유 것 필수 것 일 사람 때 감초',
  '의학 감초 성질',
  '재료 융합 시너지 효과 최대 한약 때 감초',
  '종합 감초 부작용 것 부작용 경우 대부분 양 복용 기저 질환 동반 사례 한국 전문가 한의사 처 방하 일반 복용 수준 안전 것 의학 결론',
  '일반인 임의 대량 장기간 복용 것 위험',
  '감초 부작용 원인 글리시리진 제거 게',
  '위장병 경우 식전 복용 효과',
  '일반 사용 갈근탕 엑스 과립 제제 감초 포',
  '포 복용 기준 일 수준',
  '밀리그램 그램 차지 감초 글리시리진 산 분류 그램 밀리그램 가량',
  '포 감초 규격 기준 땐 밀리그램 이 하루 포 기준 밀리그램 가량',
  '한약 하루 글리시리진 산 그램 이상 섭취',
  '일반 사용 갈근탕 엑스 과립 제제 감초 포',
  '포 복용 기준 일 수준',
  '밀리그램 그램 차지 감초 글리시리진 산 분류 그램 밀리그램 가량',
  '포 감초 규격 기준 땐 밀리그램 이 하루 포 기준 밀리그램 가량',
  '한약 하루 글리시리진 산 그램 이상 섭취',
  '이 엑스 산제 한

In [60]:
tfidf = cal_tfidf(nouns_paras)

c:\Users\ASUS\anaconda3\envs\intern\lib\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)
